In [9]:
import csv
import requests
from unidecode import unidecode 
import os

In [10]:
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"f810540cc3"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera GX";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0'
}
headers['If-Modified-Since'] = 'Sun, 19 Nov 2023 00:00:00 GMT'

In [12]:
players_id_file_src = os.path.join('..', '..', 'data', 'transfermarkt', 'players.csv')
players_list = []

with open(players_id_file_src, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
        
    for row in csv_reader:
        name = row['name']
        first_name, last_name = name.split()
        slug = f"{first_name}-{last_name}"
        
        id_transfermarkt = row['id']
        
        response = requests.get(f'https://api.sofascore.com/api/v1/search/{slug}', headers=headers)
        player = response.json()
        players_details = player.get("results", [])
        for player_data in players_details[:1]:
            player = player_data.get("entity", {})
            id_sofascore = player.get("id", "")
            player_ids = {'slug': slug,'id_transfermarkt': id_transfermarkt,'id_sofascore': id_sofascore}
            players_list.append(player_ids)
            

In [13]:
output_filename = os.path.join('..', '..', 'data', 'players', 'players_ids.csv')
with open(output_filename, 'w', newline='') as output_csv_file:
    fieldnames = ['slug', 'id_transfermarkt', 'id_sofascore']
    csv_writer = csv.DictWriter(output_csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(players_list)
            